This notebook translates GTFS to JSON that Joey finds useful.

In [1]:
import json

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [3]:
stop_times = pd.read_csv('../data/gtfs/stop_times.txt')

In [4]:
trips = {}
for trip_id, rows in stop_times.groupby('trip_id'):
    trips[trip_id] = {
        'stops': [int(i) for i in rows.stop_id.values],
        'times': [int(i) for i in pd.to_timedelta(rows.departure_time.values).seconds]}
    assert len(trips[trip_id]['stops']) == len(trips[trip_id]['times'])

In [ ]:
with open('../data/stop_times.json', 'w') as f:
    json.dump(trips, f)

In [6]:
stops = pd.read_csv('../data/gtfs/stops.txt')

In [7]:
stops_json = {}
for _, row in stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].iterrows():
    stops_json[row.stop_id] = {
        'name': row.stop_name,
        'lat': float(row.stop_lat),
        'lon': float(row.stop_lon),
    }

In [ ]:
with open('../data/stops.json', 'w') as f:
    json.dump(stops_json, f)

By the way, here are the bus counts (old, new, long, blue, pride):

In [8]:
bus_counts = np.array((93,261,33,20,2))
bus_counts / bus_counts.sum()

array([0.22738386, 0.63814181, 0.0806846 , 0.04889976, 0.00488998])

We also need to map trips to routes. We'll produce trip_id -> route_id + direction_id, and also route_id -> direction_id -> headsign.

In [20]:
trips = pd.read_csv('../data/gtfs/trips.txt')

In [27]:
trip_to_route = {}
route_to_headsign = {}
for _, trip in trips.iterrows():
    route_id = trip.route_id[:-6]  # chop off -10567 as this is always the same
    assert trip.route_id[-6:] == '-10567'
    trip_to_route[trip.trip_id] = {'route': route_id, 'dir': trip.direction_id}
    if route_id in route_to_headsign:
        route_to_headsign[route_id][trip.direction_id] = trip.trip_headsign
    else:
        route_to_headsign[route_id] = {trip.direction_id: trip.trip_headsign}

In [30]:
with open('../data/routes.json', 'w') as f:
    json.dump({
        'trips': trip_to_route,
        'routes': route_to_headsign,
    }, f)